<a href="https://colab.research.google.com/github/vamsikrishna00466/EDA/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#back test functions


In [ ]:
pip install mysql-connector-python-rf

In [ ]:
#importing relevant libraries
import os
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import operator
import decimal
import time
import mysql.connector as mysql
from math import floor
import itertools

In [ ]:
#Creating a function to convert x min data to y min data.(x<y)
def Change_Timeframe(data, timedelta):
    """
    This function will convert 5min data to 15min data.
    Parameters:
    data: Cleaned data which contains a column named 'DateTime'.
    timedelta: New timeframe in which you want to convert the data to.
    """
    
    
    #We need to sort the index before resampling
    data = data.copy()
    data.set_index(data.DateTime,inplace=True)
    data.sort_index(inplace=True)

    #Creating a Dictionary to apply functions to various columns on resampling
    aggregation_dict = {'script_name':'first',
                        'open': 'first',
                        'high': 'max',
                        'low': 'min',
                        'close': 'last',}

    #Creating a Dictionary to change the names of the respective Columns
    rename_dict = {'open':'Open 5',
                   'high':'High 5',
                   'low':'Low 5',
                   'close':'Close 5',
                   'script_name':'ScriptName'}

    return (data
            .resample(timedelta)
            .agg(aggregation_dict)
            .rename(columns=rename_dict)
            .dropna()
            .reset_index()
    )

In [ ]:
def TrueRange(data):
    data = data.copy()
    data["TR"] = np.nan
    for i in range(1,len(data)):
        h = data.loc[i,"High 5"]
        l = data.loc[i,"Low 5"]
        pc = data.loc[i-1,"Close 5"]
        
        x = h-l
        y = abs(h-pc)
        z = abs(l-pc)
        
        TR = max(x,y,z)
        data.loc[i,"TR"] = TR
    
    return data

In [ ]:
def average_true_range(data, period=18, drop_tr=True, smoothing="SMA"):
    """
    Average True Range
    :param data: Pandas Dataframe
    :param period: Period for which the ATR needs to be calculated
    :param drop_tr: Whether to drop the TR field
    :param smoothing: Smoothing type - Possible values: RMA, SMA, EMA
    :return: Pandas Dataframe with new column atr_<period>_<smoothing>. Ex: atr_14_sma
    """
    data = data.copy()
    if smoothing == "RMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].ewm(com=period - 1,
                                                                           min_periods=period).mean()
    elif smoothing == "SMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].rolling(window=period).mean()
    elif smoothing == "EMA":
        data['atr_' + str(period) + '_' + str(smoothing)] = data['TR'].ewm(span=period, adjust=False).mean()
    
    if drop_tr:
        data.drop(['TR'], inplace=True, axis=1)
    data = data.round(decimals=2)

    return data

In [ ]:
def Calmar(data):
    BeforeCostCalmar = decimal.Decimal(data.groupby("Year")["PnL"].sum().mean())/abs(min(data["DrawDown"]))
    AfterCostCalmar = decimal.Decimal(data.groupby("Year")["After Cost PnL"].sum().mean())/abs(min(data["After Cost DrawDown"]))
    return BeforeCostCalmar, AfterCostCalmar

In [ ]:
def max_runs_of_ones(bits):
    maxvalue=0
    for bit, group in itertools.groupby(bits):
        if bit: 
            maxvalue=max(maxvalue,sum(group))
    return maxvalue

In [ ]:
#Customized function for Tuning ATR_Period, Balanced Entry Time
def BackTest(data,Multiplier1=2.5, Multiplier2=0.5, Cap=1000000, RiskPercentage=0.02, ATR_Period=18, BalancedEntryTime="13:00:00", ImbalancedEntryTime="10:00:00"):
    
    Multiplier1 = decimal.Decimal(Multiplier1)
    Multiplier2 = decimal.Decimal(Multiplier2)

    def PDL(data,date):
        """
        A function to calculate Previous Day's Low Price.
        
        Parameters:
        data: Data of a particular stock.
        date: Date with respect to which you want Previous Day's Low Price.
        """
        Y_date = data.Date.unique()[list(data.Date.unique()).index(date)-1]
        Y_data = data.loc[data["Date"]==Y_date,:]
        low = min(Y_data["Low 5"])
        return decimal.Decimal(low)
    
    def PDH(data,date):
        """
        A function to calculate Previous Day's High Price.
        
        Parameters:
        data: Data of a particular stock.
        date: Date with respect to which you want Previous Day's High Price.
        """
        Y_date = data.Date.unique()[list(data.Date.unique()).index(date)-1]
        Y_data = data.loc[data["Date"]==Y_date,:]
        high= max(Y_data["High 5"])
        return decimal.Decimal(high)


    def BalancedEntry(date_df, Period = ATR_Period):
        
        Trade = pd.DataFrame(columns=["Date","Entry Type","Trade Type","Bought At","Sold At","BuyTime","SellTime","Initial ATR","Initial StopLoss","Final ATR","Final StopLoss","Exit Type","Profit/Loss"])
        DayHigh = max(date_df.loc[d["Time"]<=(datetime.datetime.strptime(BalancedEntryTime,"%H:%M:%S")-datetime.timedelta(minutes=5)).time(),"High 5"])
        DayLow = min(date_df.loc[d["Time"]<=(datetime.datetime.strptime(BalancedEntryTime,"%H:%M:%S")-datetime.timedelta(minutes=5)).time(),"Low 5"])
        LongSale=False
        ShortSale=False
        StopLoss = 0
        StopLosses = []
        for i in range(len(date_df)):
            if (LongSale==False) & (ShortSale==False) & (date_df.loc[i,"Time"]>=datetime.datetime.strptime(BalancedEntryTime, '%H:%M:%S').time()) & (date_df.loc[i,"Time"]<datetime.datetime.strptime('15:00:00', '%H:%M:%S').time()):
                if (date_df.loc[i,"High 5"]>=DayHigh):
                    BuyAt = DayHigh
                    BuyTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"\nExecuted a Long Sale on BankNifty at {BuyAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}")
                    ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                    StopLoss = max((BuyAt - Multiplier1*ATR), DayLow)
                    InitialATR = ATR
                    InitialStopLoss = StopLoss
                    LongSale=True
                    continue

                elif (date_df.loc[i,"Low 5"]<=DayLow):
                    SoldAt = DayLow
                    SellTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"\nExecuted a Short Sale on Bank Nifty at {SoldAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}")
                    ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                    StopLoss = min(SoldAt + Multiplier1*ATR, DayHigh)
                    InitialATR = ATR
                    InitialStopLoss = StopLoss
                    ShortSale=True
                    continue

            
            #If Time==15:10:00, we would need to square off our trade:
            if (date_df.loc[i,"Time"]>=datetime.datetime.strptime('15:10:00', '%H:%M:%S').time()):
                #If we have made a Long Sale, we will square it off:
                if (LongSale==True):
                    SoldAt = date_df.loc[i,"Close 5"]
                    SellTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"Exitted(3:15:00PM) the Long Sale by Selling Bank Nifty at {SoldAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    LongSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date_df.loc[i,"Date"],
                                          "Entry Type":"Balanced Entry",
                                          "Trade Type":"Long Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"15:15:00 Exit",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue

                #If we have made a Short Sale, we will square it off:
                elif (ShortSale==True):
                    BuyAt = date_df.loc[i,"Close 5"]
                    BuyTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"Exitted(3:15:00PM) the Short Sale by Buying Bank Nifty at {BuyAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    ShortSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date_df.loc[i,"Date"],
                                          "Entry Type":"Balanced Entry",
                                          "Trade Type":"Short Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"15:15:00 Exit",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue

            #If we have made a Long Sale, and the current closing price hits StopLoss, Square it off
            if (LongSale==True) & (date_df.loc[i,"Low 5"]<=StopLoss):
                if (date_df.loc[i,"High 5"]>=StopLoss):
                    SoldAt = StopLoss
                    SellTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"*StopLoss Triggered*: Exitted the Long Sale at {SoldAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    LongSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date_df.loc[i,"Date"],
                                          "Entry Type":"Balanced Entry",
                                          "Trade Type":"Long Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"StopLoss Triggered",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue
                
                elif (date_df.loc[i,"High 5"]<StopLoss):
                    SoldAt = date_df.loc[i,"High 5"]
                    SellTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"*StopLoss Triggered*: Exitted the Long Sale at {SoldAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    LongSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date_df.loc[i,"Date"],
                                          "Entry Type":"Balanced Entry",
                                          "Trade Type":"Long Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"StopLoss Triggered",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue

            #If we have made a Short Sale, and the current closing price hits StopLoss, Square it off
            elif (ShortSale==True) & (date_df.loc[i,"High 5"]>=StopLoss):
                if (date_df.loc[i,"Low 5"]<=StopLoss):
                    BuyAt = StopLoss
                    BuyTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"*StopLoss Triggered*: Exitted the Short Sale at {BuyAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    ShortSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date,
                                          "Entry Type":"Balanced Entry",
                                          "Trade Type":"Short Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"StopLoss Triggered",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue
                
                elif (date_df.loc[i,"Low 5"]>StopLoss):
                    BuyAt = date_df.loc[i,"Low 5"]
                    BuyTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"*StopLoss Triggered*: Exitted the Short Sale at {BuyAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    ShortSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date,
                                          "Entry Type":"Balanced Entry",
                                          "Trade Type":"Short Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"StopLoss Triggered",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue
                   
            #If Time<=15:10:00, and we have executed any LongSale or ShortSale, we would need to update our StopLoss:
            if (date_df.loc[i,"Time"]<=datetime.datetime.strptime('15:10:00', '%H:%M:%S').time()):
                if (date_df.loc[i,"Time"]<datetime.datetime.strptime('15:00:00', '%H:%M:%S').time()):
                    if (LongSale==True):
                        ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                        StopLosses.append(InitialStopLoss)
                        StopLosses.append(BuyAt - Multiplier1*ATR)
                        StopLoss = max(max(StopLosses), DayLow)
                    elif (ShortSale==True):
                        ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                        StopLosses.append(InitialStopLoss)
                        StopLosses.append(SoldAt + Multiplier1*ATR)
                        StopLoss = min(min(StopLosses), DayHigh)
                
                elif (date_df.loc[i,"Time"]>=datetime.datetime.strptime('15:00:00', '%H:%M:%S').time()):
                    if (LongSale==True):
                        ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                        StopLosses.append(InitialStopLoss)
                        StopLosses.append(date_df.loc[i-1,"Close 5"] - Multiplier2*ATR)
                        StopLoss = max(max(StopLosses), DayLow)
                    elif (ShortSale==True):
                        ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                        StopLosses.append(InitialStopLoss)
                        StopLosses.append(date_df.loc[i-1,"Close 5"] + Multiplier2*ATR)
                        StopLoss = min(min(StopLosses), DayHigh)
        
        return Trade


    def ImbalancedEntry(date_df, Period=ATR_Period):
        
        Trade = pd.DataFrame(columns=["Date","Entry Type","Trade Type","Bought At","Sold At","BuyTime","SellTime","Initial ATR","Initial StopLoss","Final ATR","Final StopLoss","Exit Type","Profit/Loss"])
        DayHigh = max(date_df.loc[d["Time"]<=(datetime.datetime.strptime(ImbalancedEntryTime,"%H:%M:%S")-datetime.timedelta(minutes=5)).time(),"High 5"])
        DayLow = min(date_df.loc[d["Time"]<=(datetime.datetime.strptime(ImbalancedEntryTime,"%H:%M:%S")-datetime.timedelta(minutes=5)).time(),"Low 5"])
        LongSale=False
        ShortSale=False
        StopLoss = 0
        StopLosses = []
        for i in range(len(date_df)):
            if (LongSale==False) & (ShortSale==False) & (date_df.loc[i,"Time"]>=datetime.datetime.strptime(ImbalancedEntryTime, '%H:%M:%S').time()) & (date_df.loc[i,"Time"]<datetime.datetime.strptime('15:00:00', '%H:%M:%S').time()):
                if (date_df.loc[i,"High 5"]>=DayHigh):
                    BuyAt = decimal.Decimal(DayHigh)
                    BuyTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"\nExecuted a Long Sale on BankNifty at {BuyAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}")
                    ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                    StopLoss = max(BuyAt - Multiplier1*ATR, DayLow)
                    InitialATR = ATR
                    InitialStopLoss = StopLoss
                    LongSale=True
                    continue

                elif (date_df.loc[i,"Low 5"]<=DayLow):
                    SoldAt = DayLow
                    SellTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"\nExecuted a Short Sale on Bank Nifty at {SoldAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}")
                    ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                    StopLoss = min(SoldAt + Multiplier1*ATR, DayHigh)
                    InitialATR = ATR
                    InitialStopLoss = StopLoss
                    ShortSale=True
                    continue
            
            #If Time==15:10:00, we would need to square off our trade:
            if (date_df.loc[i,"Time"]>=datetime.datetime.strptime('15:10:00', '%H:%M:%S').time()):
                #If we have made a Long Sale, we will square it off:
                if (LongSale==True):
                    SoldAt = date_df.loc[i,"Close 5"]
                    SellTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"Exitted(3:15:00PM) the Long Sale by Selling Bank Nifty at {SoldAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    LongSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date_df.loc[i,"Date"],
                                          "Entry Type":"Imbalanced Entry",
                                          "Trade Type":"Long Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"15:15:00 Exit",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue

                #If we have made a Short Sale, we will square it off:
                elif (ShortSale==True):
                    BuyAt = date_df.loc[i,"Close 5"]
                    BuyTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"Exitted(3:15:00PM) the Short Sale by Buying Bank Nifty at {BuyAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    ShortSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date_df.loc[i,"Date"],
                                          "Entry Type":"Imbalanced Entry",
                                          "Trade Type":"Short Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"15:15:00 Exit",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue

            #If we have made a Long Sale, and the current closing price hits StopLoss, Square it off
            if (LongSale==True) & (date_df.loc[i,"Low 5"]<=StopLoss):
                if (date_df.loc[i,"High 5"]>=StopLoss):
                    SoldAt = StopLoss
                    SellTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"*StopLoss Triggered*: Exitted the Long Sale at {SoldAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    LongSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date_df.loc[i,"Date"],
                                          "Entry Type":"Imbalanced Entry",
                                          "Trade Type":"Long Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"StopLoss Triggered",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue
                
                elif (date_df.loc[i,"High 5"]<StopLoss):
                    SoldAt = date_df.loc[i,"High 5"]
                    SellTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"*StopLoss Triggered*: Exitted the Long Sale at {SoldAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    LongSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date_df.loc[i,"Date"],
                                          "Entry Type":"Imbalanced Entry",
                                          "Trade Type":"Long Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"StopLoss Triggered",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue

            #If we have made a Short Sale, and the current closing price hits StopLoss, Square it off
            elif (ShortSale==True) & (date_df.loc[i,"High 5"]>=StopLoss):
                if (date_df.loc[i,"Low 5"]<=StopLoss):
                    BuyAt = StopLoss
                    BuyTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"*StopLoss Triggered*: Exitted the Short Sale at {BuyAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    ShortSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date,
                                          "Entry Type":"Imbalanced Entry",
                                          "Trade Type":"Short Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"StopLoss Triggered",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue
                
                elif (date_df.loc[i,"Low 5"]>StopLoss):
                    BuyAt = date_df.loc[i,"Low 5"]
                    BuyTime = date_df.loc[i,"Time"]
                    date = date_df.loc[i,"Date"]
                    print(f"*StopLoss Triggered*: Exitted the Short Sale at {BuyAt} on {datetime.datetime.strptime(date, '%Y-%m-%d'):%B %d, %Y} at {date_df.loc[i,'Time'].strftime('%I:%M:%S %p')}\n")
                    ShortSale = False
                    StopLosses = []
                    Trade = Trade.append({"Date":date,
                                          "Entry Type":"Imbalanced Entry",
                                          "Trade Type":"Short Sale",
                                          "Bought At":BuyAt,
                                          "Sold At":SoldAt,
                                          "BuyTime":BuyTime,
                                          "SellTime":SellTime,
                                          "Initial ATR":InitialATR,
                                          "Initial StopLoss":InitialStopLoss,
                                          "Final ATR":ATR,
                                          "Final StopLoss":StopLoss,
                                          "Exit Type":"StopLoss Triggered",
                                          "Profit/Loss":(SoldAt-BuyAt)},
                                         ignore_index=True)
                    continue
                        
            #If Time<=15:10:00, and we have executed any LongSale or ShortSale, we would need to update our StopLoss:
            if (date_df.loc[i,"Time"]<=datetime.datetime.strptime('15:10:00', '%H:%M:%S').time()):
                if (LongSale==True):
                    ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                    StopLosses.append(InitialStopLoss)
                    StopLosses.append(date_df.loc[i-1,"Close 5"] - Multiplier1*ATR)
                    StopLoss = max(max(StopLosses), DayLow)
                elif (ShortSale==True):
                    ATR = decimal.Decimal(date_df.loc[i-1,f"atr_{Period}_SMA"])
                    StopLosses.append(InitialStopLoss)
                    StopLosses.append(date_df.loc[i-1,"Close 5"] + Multiplier1*ATR)
                    StopLoss = min(min(StopLosses), DayHigh)
        
        return Trade
    
    
    
    def PnL(data, Capital=Cap, Risk=RiskPercentage):
        data = data.copy()
        RiskedCapital = decimal.Decimal(Capital*Risk)
        data["Year"] = list(map(lambda x: x[:4],data["Date"]))
        data["DistanceToInitialStopLoss"] = 0
        for i in range(len(data)):
            if data.loc[i,"Trade Type"]=="Long Sale":
                data.loc[i,"DistanceToInitialStopLoss"] = data.loc[i,"Bought At"] - data.loc[i,"Initial StopLoss"]
            elif data.loc[i,"Trade Type"]=="Short Sale":
                data.loc[i,"DistanceToInitialStopLoss"] = data.loc[i,"Initial StopLoss"] - data.loc[i,"Sold At"]
        data["Quantity"] = list(map(lambda x: floor((RiskedCapital/x)/25)*25,data["DistanceToInitialStopLoss"]))
        data["Lots"] = list(map(lambda x: x/25,data["Quantity"]))
        data["PnL"] = data["Profit/Loss"]*data["Quantity"]
        data["Cumulative"] = data["PnL"].cumsum()
        data["HighValue"] = data["Cumulative"].cummax()
        data["DrawDown"] = data["Cumulative"] - data["HighValue"]
        data["Cost/Stock"] = 0
        for i in range(len(data)):
            if data.loc[i,"Trade Type"]=="Long Sale":
                data.loc[i,"Cost/Stock"] = data.loc[i,"Bought At"]*decimal.Decimal(0.0004)
            elif data.loc[i,"Trade Type"]=="Short Sale":
                data.loc[i,"Cost/Stock"] = data.loc[i,"Sold At"]*decimal.Decimal(0.0004)
        data["Cost/Lot"] = data["Cost/Stock"]*data["Quantity"]
        data["After Cost PnL"] = data["PnL"] - data["Cost/Lot"]
        data["After Cost Cumulative"] = data["After Cost PnL"].cumsum()
        data["After Cost HighValue"] = data["After Cost Cumulative"].cummax()
        data["After Cost DrawDown"] = data["After Cost Cumulative"] - data["After Cost HighValue"]
        
        data = data[['Date', 'Year', 'Entry Type', 'Trade Type', 'Bought At', 'Sold At', 'BuyTime', 'SellTime', 'Initial ATR', 'Initial StopLoss', 'Final ATR', 'Final StopLoss', 'Exit Type', 'DistanceToInitialStopLoss', 'Profit/Loss', 'Quantity', 'Lots', 'PnL', 'Cumulative', 'HighValue', 'DrawDown', 'Cost/Stock', 'Cost/Lot', 'After Cost PnL', 'After Cost Cumulative', 'After Cost HighValue', 'After Cost DrawDown']]
        return data
    
    


    Trades = pd.DataFrame(columns=["Date","Entry Type","Trade Type","Bought At","Sold At","BuyTime","SellTime","Initial ATR","Initial StopLoss","Final ATR","Final StopLoss","Exit Type","Profit/Loss"])
    
    
    for i in data.Date.unique()[1:]:
        d = data.loc[data["Date"]==i]
        d = d.reset_index(drop=True)
        PD_High = PDH(data,i)
        PD_Low = PDL(data,i)
        First30MinClose = d.loc[d["Time"]==datetime.datetime.strptime("09:40:00",'%H:%M:%S').time(),"Close 5"].values
        if (PD_Low<=First30MinClose) & (First30MinClose<=PD_High):
            Trades = Trades.append(BalancedEntry(d),ignore_index=True)
        if (PD_Low>First30MinClose) or (First30MinClose>PD_High):
            Trades = Trades.append(ImbalancedEntry(d),ignore_index=True)
    
    Trades = PnL(Trades)
    Trades["Year-Month"] = list(map(lambda x: x[:7],Trades["Date"]))
    Trades["S.No."] = list(range(1,len(Trades)+1))
    Trades = Trades[['S.No.', 'Date', 'Year', 'Year-Month', 'Entry Type', 'Trade Type', 'Bought At', 'Sold At', 'BuyTime', 'SellTime', 'Initial ATR', 'Initial StopLoss', 'Final ATR', 'Final StopLoss', 'Exit Type', 'DistanceToInitialStopLoss', 'Profit/Loss', 'Quantity', 'Lots', 'PnL', 'Cumulative', 'HighValue', 'DrawDown', 'Cost/Stock', 'Cost/Lot', 'After Cost PnL', 'After Cost Cumulative', 'After Cost HighValue', 'After Cost DrawDown']]
    return Trades

In [ ]:
########################## Connecting to the Database ##########################
# enter your server IP address/domain name
HOST = "host" # or "domain.com"
# database name, if you want just to connect to MySQL server, leave it empty
DATABASE = "dbname"
# this is the user you create
USER = "name"
# user password
PASSWORD = "pass"
# connect to MySQL server
db_connection = mysql.connect(host=HOST, user=USER, password=PASSWORD)
print("Connected to:", db_connection.get_server_info())
cursor = db_connection.cursor()     # get the cursor

#cursor.execute("SELECT * FROM instrument_details")



cursor.execute("use stock_production;")
cursor.execute(f"SELECT id,script_name,start_date,start_time,open,high,low,close FROM cash_instruments WHERE start_date BETWEEN DATE('2019-01-01') AND DATE('2020-12-31');") 
myresult = cursor.fetchall()
df = pd.DataFrame(myresult, columns=["id","script_name","start_date","start_time","open","high","low","close"])
df = df.drop(index = df[df["script_name"]=="NIFTY"].index).reset_index(drop=True)
df["DateTime"] = list(map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'),[i+j[6:] for (i,j) in zip(list(map(str,df.start_date)),list(map(str,df.start_time)))]))
df.drop(columns = ["id","start_date","start_time"],inplace=True)
new_df = Change_Timeframe(df,"5Min")
new_df["Date"] = list(map(lambda x: str(x.date()),new_df.DateTime))
new_df["Time"] = list(map(lambda x: x.time(),new_df.DateTime))
new_df = new_df.drop(columns="DateTime").copy()
new_df = TrueRange(new_df)

In [ ]:
new_df.head()

In [ ]:
ATR_df = average_true_range(data = new_df, period=18, drop_tr=True, smoothing="SMA")
Trades = BackTest(ATR_df)
Before, After = Calmar(Trades)
Before = round(Before,2)
After = round(After,2)
Trades.to_excel(f"Strategy 3 v1.08.xlsx",index=False)
NoOfMonthsNegative = len(list(filter(lambda x: x<0,Trades.groupby("Year-Month")["After Cost PnL"].sum())))/len(Trades.groupby("Year-Month")["After Cost PnL"].sum())
AnnualProfit = float(sum(list(filter(lambda x: x>0,Trades["After Cost PnL"])))/len(Trades["Year"].unique()))
AnnualLoss = float(sum(list(filter(lambda x: x<0,Trades["After Cost PnL"])))/len(Trades["Year"].unique()))
MaxConsecutiveProfit = max_runs_of_ones(Trades["After Cost PnL"]>0)
MaxConsecutiveLoss = max_runs_of_ones(Trades["After Cost PnL"]<0)
DrawDownTime = len(Trades)
CalmarRatio = Trades.groupby("Year")["After Cost PnL"].sum().mean()/float(abs(min(Trades["After Cost DrawDown"])))

In [ ]:
df = pd.read_excel("/content/Strategy 3 v1.08.xlsx")

In [ ]:
df.head()